In [109]:
import pandas as pd
import plotly.express as px
from collections import Counter

# Add the parent directory of the 'utils' directory to the Python path
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import utils.utils as u

### Expand dataset for multiple owners

#### Load dataset

In [110]:
df = pd.read_json('../data_catastici/data_post-processing/pipeline_steps/catastici_1741_STD.json')
df['uidx'] = df['uidx'].astype(str)
df.head()

,uidx,id,owner_name,owner_code,owner_count,owner_count_remark,owner_entity,owner_entity_group,owner_first_name,owner_family_name,...,id_napo,quantity_income,quality_income,author,place,parish,sestiere,uid,path_img,geometry
0,0,1,Liberal Campi secondo prete titolato della Chiesa,PPL,1,,,,Liberal,CAMPI,...,3052,nan,nan,Davide,Campo vicino alla Chiesa,San Cancian,CN,CNC-0001,436_Cannaregio/11_SCancian/SCancian_0_1.png,POINT (291832.8461721611 5035321.328300701)
1,1,2,Filippo Frari terzo prete titolato della Chiesa,PPL,1,,,,Filippo,FRARI,...,3051,nan,nan,Davide,Campo vicino alla Chiesa,San Cancian,CN,CNC-0002,436_Cannaregio/11_SCancian/SCancian_0_1.png,POINT (291841.5307544547 5035307.748867494)
2,2,3,Pievan di San Cancian,ent_REL_TTL,1,,PIEVAN DI SAN CANCIAN,CHIESA DI SAN CANCIANO,,,...,nan,nan,nan,Davide,Campo vicino alla Chiesa,San Cancian,CN,CNC-0003,436_Cannaregio/11_SCancian/SCancian_0_1.png,POINT (291845.4782538932 5035313.512255858)
3,3,4,Filippo Frari terzo prete titolato della Chiesa,PPL,1,,,,Filippo,FRARI,...,nan,nan,nan,Davide,Campo vicino alla Chiesa,San Cancian,CN,CNC-0004,436_Cannaregio/11_SCancian/SCancian_0_1.png,POINT (291846.4256626792 5035312.091152506)
4,4,5,Agostin Filippi di Vicenza,PPL,1,,,,Agostin,FILIPPI,...,nan,nan,nan,Davide,Campo vicino alla Chiesa,San Cancian,CN,CNC-0005,436_Cannaregio/11_SCancian/SCancian_0_1.png,POINT (291824.082739216 5035303.564443278)


#### Load necessary dictionaries

In [111]:
first_names = sorted(pd.read_json('../dictionaries/PPL_dictionary/first_names.json')[0].to_list(), key=len, reverse=True)
unknown_relatives_plur = sorted(pd.read_json('../dictionaries/PPL_dictionary/unknown_relatives_plur.json')[0].to_list(), key=len, reverse=True)

#### Expand dataset - one row per owner

In [112]:
owner_count_index = df.columns.get_loc("owner_count")
df.insert(owner_count_index, "parcel_portion", 1)

In [113]:
def expand_rows(row, columns_with_pipe):
    expanded_rows = []
    
    reference_col = 'owner_family_group' if len(row['owner_family_group']) > 0 else 'owner_entity_group'
    num_owners = row[reference_col].count('|') + 1
    
    for i in range(num_owners):
        new_row = row.copy()
        
        for col in columns_with_pipe:
            split_values = row[col].split('|')
            if " | " in row[col] and len(split_values) < num_owners:
                split_values += ['-'] * (num_owners - len(row[col].split('|')))
            
            if i < len(split_values):
                new_row[col] = u.remove_extra_spaces(split_values[i])
        
        # assign new uid
        new_row['uidx'] = f"{new_row['uidx']}_{i}"
        
        # assign parcel portion
        new_row['parcel_portion'] = round(1.0 / num_owners, 2)
        
        # adjust title
        if len(new_row.get('owner_title_std', '')) > 0 and new_row.get('owner_title', '') == '-':
            new_row['owner_title'] = new_row.get('owner_title_std', '')
        
        # adjust owner count
        if new_row.get('owner_first_name', '').replace('_', '') in unknown_relatives_plur:
            new_row['owner_count'] = 2
            new_row['owner_count_remark'] = '2+'
        else:
            new_row['owner_count'] = 1
            new_row['owner_count_remark'] = ''
            
        expanded_rows.append(new_row)
        
    return expanded_rows

In [114]:
df_filtered = df[df.apply(lambda x: (isinstance(x['owner_family_group'], str) and len(x['owner_family_group']) > 0 and '|' in x['owner_family_group']) or 
                                      (isinstance(x['owner_entity_group'], str) and len(x['owner_entity_group']) > 0 and '|' in x['owner_entity_group']), axis=1)]


columns_with_pipe = [col for col in df_filtered.columns if df_filtered[col].astype(str).str.contains(' | ', regex=False).any()]

expanded_parcels = []
df_filtered.apply(lambda row: expanded_parcels.extend(expand_rows(row, columns_with_pipe)), axis=1)
df_expanded = pd.DataFrame(expanded_parcels)


In [115]:
df['uidx'] = df['uidx'].astype(str)

new_rows_list = []
for index, row in df.iterrows():
    original_uidx = row['uidx']
    matching_expanded_rows = df_expanded[df_expanded['uidx'].str.startswith(original_uidx + "_")]
    
    if not matching_expanded_rows.empty:
        new_rows_list.extend(matching_expanded_rows.to_dict('records'))
    else:
        new_rows_list.append(row.to_dict())

df_expanded_merged = pd.DataFrame(new_rows_list)
df_expanded_merged['uidx'] = df_expanded_merged['uidx'].astype(str)


In [116]:
df_expanded_merged.to_json('../data_catastici/data_post-processing/pipeline_steps/catastici_1741_STD_expanded.json', orient='records')

In [117]:
with pd.ExcelWriter('../data_catastici/data_post-processing/catastici_1741_STD_expanded.xlsx') as writer:
    df_expanded_merged.to_excel(writer, sheet_name='catastici_1741') 

### Distribution of Owner Count

In [118]:
owner_count_counts = df['owner_count'].value_counts().reset_index()
owner_count_counts.columns = ['owner_count', 'count']

fig = px.bar(owner_count_counts, x='owner_count', y='count', title='Distribution of Owner Counts')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


### Distribution of Owner Codes

#### Not assigned or to-verify entries

In [119]:
filtered_df = df[df['owner_code'].str.contains('VRF') | (df['owner_code'] == 'TODO')].copy()

owner_code_counts = filtered_df['owner_code'].value_counts().reset_index()
owner_code_counts.columns = ['owner_code', 'count']

fig = px.bar(owner_code_counts, x='owner_code', y='count', title='Distribution of unassigned or to-verify entries')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


#### The following entries have to be verified by hand

In [120]:
filtered_df = df[df['owner_code'].str.contains('VRF') | (df['owner_code'] == 'TODO')].copy()
filtered_df.loc[:, 'owner_name'] = filtered_df['owner_name'].apply(lambda x: u.text_to_minimal(x))
unique_owner_names = filtered_df['owner_name'].unique()

for i, owner_name in enumerate(unique_owner_names):
    print(i+1, ": ", owner_name)

1 :  francesco maria segalla da vicenza
2 :  demetrio conte
3 :  giovanni battista e lodovico calice q. bortolo
4 :  santa sporini relicta quondam giacomo marchi
5 :  bernardi alessio ruini
6 :  nobil homo gerolamo lion cavazza
7 :  conte bertoli
8 :  luganeghera in barbaria delle tavole
9 :  luganeghera in barbaria dalle tavole
10 :  nobil homo conte zuanne vidiman
11 :  conte antonio e fratello bortoli
12 :  signor conte antonio bartoli
13 :  nobili homini conti cesare e nipoti piovene
14 :  nobili homini ser antonio zorzi, conte lodovico manin, ser piero girolamo capello
15 :  illustrissima signora anna maria marcello piceni
16 :  nobil homo ser conte zuanne vidiman
17 :  nobil homo ser girolamo michiel lini
18 :  nobil homo ser marco michiel salamon
19 :  nobil homo ser conte lodovico manin
20 :  ill.mo conte francesco gerardini
21 :  antonio maria giani bernardini
22 :  nobil donna fiorenza lion marcello
23 :  conte girolamo benzon
24 :  conte ferro da padova
25 :  conte cucina pa

#### Distribution of all owner codes
Except unassigned and to-verify

In [121]:
filtered_df = df[~df['owner_code'].str.contains('VRF') & (df['owner_code'] != 'TODO')].copy()

print(filtered_df['owner_code'].value_counts())

owner_code_counts = filtered_df['owner_code'].value_counts().reset_index()
owner_code_counts.columns = ['owner_code', 'count']

fig = px.bar(owner_code_counts, x='owner_code', y='count', title='Distribution of Owner Codes')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


owner_code
PPL                          22462
PPL_m                         4331
ent_REL                       1636
ent_SCL_GRD                    963
ent_REL_TTL                    901
ent_VNZ                        325
ent_OTH                        290
BLK                            279
ent_SCR                        265
ent_REL_TTL_UNL                179
ent_REL_UNL                    137
ent_SCR_m                       83
UNK                             75
ent_SCL_REL                     62
ent_SCL_MST                     58
ent_SCL_UNL                     57
ent_GLD                         53
ent_VNZ_TTL                     52
ent_REL_TTL_m                   41
ent_JEW                         30
ent_REL_m                       30
ent_SCL_MST_m                   10
ent_REL_UNL_m | ent_REL_m        7
ent_REL_TTL_UNL_m                6
ent_SCR_UNL                      6
ent_VNZ_UNL                      4
ent_REL_m | PPL_m                2
ent_SCL_UNL_m | PPL_m            2
ent_VNZ_m

## Distribution of Entities

In [122]:
min_occurrences = 1
num_displayed = 20

### Distribution of Entity Owner Codes

In [123]:
entity_code_mapping = {
    'ent_GLD': 'Guild entities',
    'ent_JEW': 'Jew entities',
    'ent_OTH': 'Other entities',
    'ent_REL_UNL': 'Religious *unlinked* entities',
    'ent_REL': 'Religious entities',
    'ent_REL_TTL_UNL': 'Religious Title *unlinked* entities',
    'ent_REL_TTL': 'Religious Title entities',
    'ent_SCL_UNL': 'Scuole *unlinked* entities',
    'ent_SCL_GRD': 'Scuole Grandi entities',
    'ent_SCL_MST': 'Scuole Mestieri entities',
    'ent_SCL_REL': 'Scuole Religious entities',
    'ent_SCR_UNL': 'Social Care *unlinked* entities',
    'ent_SCR': 'Social Care entities',
    'ent_VNZ': 'Repubblica di Venezia',
    'ent_VNZ_TTL': 'Title Repubblica di Venezia',
    'ent_VNZ_UNL': '*unlinked* Repubblica di Venezia'
}

In [124]:
filtered_df_ent_types = df[df['owner_code'].str.startswith('ent_')].copy()

filtered_df_ent_types.loc[:, 'owner_code'] = filtered_df_ent_types['owner_code'].map(lambda x: x[:-2] if x.endswith('_m') else x)
filtered_df_ent_types.loc[:, 'owner_legend'] = filtered_df_ent_types['owner_code'].map(entity_code_mapping)

pie_fig = px.pie(filtered_df_ent_types, names='owner_legend', title='Distribution of Entity Types (Pie Chart)')
pie_fig.update_layout(height=500)
pie_fig.show()

hist_fig = px.histogram(filtered_df_ent_types, x='owner_legend', title='Distribution of Entity Types (Histogram)')
hist_fig.update_layout(height=500)
hist_fig.update_traces(texttemplate='%{y}', textposition='inside')
hist_fig.show()


### Religious entities

#### Entities (religious)

##### Entity mentions assigned to parcels (religious)

In [125]:
filtered_df_ent_rel = df[df['owner_code'].str.startswith('ent_REL')].copy()
filtered_df_ent_rel['owner_entity_display'] = filtered_df_ent_rel['owner_entity']

entity_counts = filtered_df_ent_rel['owner_entity_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_display'}, 
             title=f'Top {num_displayed} Religious entities with more than {min_occurrences} occurrences')

fig.update_layout(height=600) 
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


##### Stadardised entities assigned to parcels (religious)

In [126]:
filtered_df_ent_rel = df[df['owner_code'].str.startswith('ent_REL')].copy()
filtered_df_ent_rel['owner_entity_group_display'] = filtered_df_ent_rel['owner_entity_group']

entity_counts = filtered_df_ent_rel['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Religious entities with more than {min_occurrences} occurrences')

fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


#### Title-owned (Religious) parametrised on rent

In [127]:
# TODO

#### Titles (religious)

##### Entities owners of the title's parcel (religious)

In [128]:
filtered_df_ent_rel = df[df['owner_code'].str.startswith('ent_REL_TTL')].copy()
filtered_df_ent_rel['owner_entity_group_display'] = filtered_df_ent_rel['owner_entity_group']

entity_counts = filtered_df_ent_rel['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} (Title owned) Religious entities with more than {min_occurrences} occurrences')

fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


##### Title mentions assigned to parcels (religious)

In [129]:
filtered_df_ent_rel = df[(df['owner_code'].str.len() > 0) & df['owner_code'].str.startswith('ent_REL_TTL')].copy()
filtered_df_ent_rel['owner_title'] = filtered_df_ent_rel['owner_title']

entity_counts = filtered_df_ent_rel['owner_title'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Religious Title mentions with more than {min_occurrences} occurrences')

fig.update_layout(height=500)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


##### Stadardised titles assigned to parcels (religious)

In [130]:
filtered_df_ent_rel = df[(df['owner_code'].str.len() > 0) & (df['owner_title_std'].str.len() > 0) & df['owner_code'].str.startswith('ent_REL_TTL')].copy()

entity_counts = filtered_df_ent_rel['owner_title_std'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} standardised Religious titles with more than {min_occurrences} occurrences')

fig.update_layout(height=500)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


### Venezia entities

#### Entities (R. di Venezia)

##### Entity mentions assigned to parcels (R. di Venezia)

In [131]:
filtered_df_ent_vnz = df[df['owner_code'].str.startswith('ent_VNZ')].copy()
filtered_df_ent_vnz['owner_entity_display'] = filtered_df_ent_vnz['owner_entity']

entity_counts = filtered_df_ent_vnz['owner_entity_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_display'}, 
             title=f'Top {num_displayed} R. di Venezia mentions with more than {min_occurrences} occurrences')

fig.update_layout(height=650)
fig.update_traces(texttemplate='%{y}', textposition='inside')
 

fig.show()


##### Stadardised entities assigned to parcels (R. di Venezia)

In [132]:
filtered_df_ent_vnz = df[df['owner_code'].str.startswith('ent_VNZ')].copy()
filtered_df_ent_vnz['owner_entity_group_display'] = filtered_df_ent_vnz['owner_entity_group']

entity_counts = filtered_df_ent_vnz['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} R. di Venezia entities with more than {min_occurrences} occurrences')

fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.update_layout(height=650)
fig.show()


#### Titles (R. di Venezia)

##### Entities owners of the title's parcel (R. di Venezia)

In [133]:
filtered_df_ent_vnz = df[df['owner_code'].str.startswith('ent_VNZ_TTL')].copy()
filtered_df_ent_vnz['owner_entity_group_display'] = filtered_df_ent_vnz['owner_entity_group']

entity_counts = filtered_df_ent_vnz['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} (Title owned) R. di Venezia entities with more than {min_occurrences} occurrences')

fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()

##### Title mentions assigned to parcels (R. di Venezia)

In [134]:
filtered_df_ent_vnz = df[(df['owner_code'].str.len() > 0) & df['owner_code'].str.startswith('ent_VNZ_TTL')].copy()
filtered_df_ent_vnz['owner_title'] = filtered_df_ent_vnz['owner_title']

entity_counts = filtered_df_ent_vnz['owner_title'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} R. di Venezia Title mentions with more than {min_occurrences} occurrences')

fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


##### Stadardised titles assigned to parcels (R. di Venezia)

In [135]:
filtered_df_ent_vnz = df[(df['owner_code'].str.len() > 0) & df['owner_code'].str.startswith('ent_VNZ_TTL')].copy()

entity_counts = filtered_df_ent_vnz['owner_title_std'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} standardised R. di Venezia titles with more than {min_occurrences} occurrences')

fig.update_layout(height=500)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


### Scuole entities

#### Distribution of Scuole types

In [136]:
scuole_df = df[df['owner_code'].str.contains('ent_SCL_')].copy()
scuole_df['Scuole Type'] = scuole_df['owner_code'].map(entity_code_mapping)

scuole_counts = scuole_df['Scuole Type'].value_counts().reset_index()
scuole_counts.columns = ['Scuole Type', 'count']

fig = px.pie(scuole_counts, names='Scuole Type', values='count', 
             title='Count of Different Types of Scuole')
fig.update_layout(height=500)
fig.show()

#### Scuole Grandi

In [137]:
scuole_df = df[df['owner_code'].str.startswith('ent_SCL_GRD')].copy()
scuole_df['owner_entity_group_display'] = scuole_df['owner_entity_group']

entity_counts = scuole_df['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Scuole Grandi with more than {min_occurrences} occurrences')

fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


#### Scuole Religious

In [138]:
scuole_df = df[df['owner_code'].str.startswith('ent_SCL_REL')].copy()
scuole_df['owner_entity_group_display'] = scuole_df['owner_entity_group']

entity_counts = scuole_df['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Scuole Religious with more than {min_occurrences} occurrences')

fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


#### Scuole Mestieri

In [139]:
scuole_df = df[df['owner_code'].str.startswith('ent_SCL_MST')].copy()
scuole_df['owner_entity_group_display'] = scuole_df['owner_entity_group']

entity_counts = scuole_df['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Scuole Mestieri with more than {min_occurrences} occurrences')

fig.update_layout(height=500)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


### Social Care and Guild Entities

#### Social Care

In [140]:
social_care_df = df[df['owner_code'].str.startswith('ent_SCR')].copy()
social_care_df['owner_entity_group_display'] = social_care_df['owner_entity_group']

entity_counts = social_care_df['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)


fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Social Care entities with more than {min_occurrences} occurrences')

fig.update_layout(height=600)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


#### Guild

In [141]:
guild_df = df[df['owner_code'].str.startswith('ent_GLD')].copy()
guild_df['owner_entity_group_display'] = guild_df['owner_entity_group']

entity_counts = guild_df['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Guild entities with more than {min_occurrences} occurrences')

fig.update_layout(height=500)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


#### Jew entities

In [142]:
jew_df = df[df['owner_code'].str.startswith('ent_JEW')].copy()
jew_df['owner_entity_group_display'] = jew_df['owner_entity_group']

entity_counts = jew_df['owner_entity_group_display'].value_counts()
filtered_entity_counts = entity_counts[entity_counts >= min_occurrences]

top_entities = filtered_entity_counts.head(num_displayed)

fig = px.bar(top_entities, x=top_entities.index, y=top_entities.values, 
             labels={'y': 'count', 'index': 'owner_entity_group_display'}, 
             title=f'Top {num_displayed} Jew entities with more than {min_occurrences} occurrences')

fig.update_layout(height=500)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()


### Misclassifications: Entities

According to most common mention (not to Entity group name).

#### Number of misclassifications in non-grouped mentions (pre-standardisation)

In [143]:
misclassification_entities_df = df[(df['owner_entity'].str.len() > 0) | (df['owner_entity_group'].str.len() > 0)].copy()

most_common_names = misclassification_entities_df.groupby('owner_entity_group')['owner_entity'].agg(
    lambda x: x.mode()[0] if not x.mode().empty else pd.NA
).reset_index().rename(columns={'owner_entity': 'most_common_name'})
misclassification_entities_df = misclassification_entities_df.merge(most_common_names, on='owner_entity_group')

misclassification_entities_df['misclassified'] = misclassification_entities_df['owner_entity'] != misclassification_entities_df['most_common_name']

group_summary = misclassification_entities_df.groupby('owner_entity_group').agg(
    total_ownerships=('owner_entity_group', 'size'),  # total occurrences of each entity group
    misclassified=('misclassified', 'sum')  # sum of misclassified instances per group
).reset_index()

group_summary['misclassification_percentage'] = (group_summary['misclassified'] / group_summary['total_ownerships']) * 100
group_summary['misclassification_percentage'] = group_summary['misclassification_percentage'].round(2)

In [144]:
group_summary_sorted_misclassified_ent = group_summary.sort_values(by='misclassified', ascending=False)

fig1 = px.bar(group_summary_sorted_misclassified_ent.head(num_displayed), x='owner_entity_group', y='misclassified',
              text='misclassified',
              labels={'misclassified': 'Number of Misclassifications', 'owner_entity_group': 'Entity Group'},
              title=f'Top {num_displayed} Entity Groups with higher number of Misclassifications')
fig1.update_traces(texttemplate='%{text}', textposition='outside')
fig1.update_layout(xaxis={'categoryorder': 'total descending'}, yaxis_title="Number of Misclassifications")


group_summary_sorted_occurrences_ent = group_summary.sort_values(by='total_ownerships', ascending=False)

fig2 = px.bar(group_summary_sorted_occurrences_ent.head(num_displayed), x='owner_entity_group', y='misclassified',
              text='misclassified',
              labels={'misclassified': 'Number of Misclassifications', 'owner_entity_group': 'Entity Group', 'total_ownerships': 'Total Ownerships'},
              title=f'Number of Misclassifications of Top {num_displayed} Entity Groups by Number of Ownerships')
fig2.update_traces(texttemplate='%{text}', textposition='outside')

fig1.update_traces(texttemplate='%{y}', textposition='inside')
fig2.update_traces(texttemplate='%{y}', textposition='inside')
fig1.update_layout(height=650)
fig2.update_layout(height=650)
fig1.show()
fig2.show()

#### Percentage (%) of misclassifications in non-grouped mentions (pre-standardisation)

In [145]:
group_summary_sorted_percentage_ent = group_summary.sort_values(by='misclassification_percentage', ascending=False)
group_summary_sorted_percentage_ent['misclassification_percentage'] = group_summary_sorted_percentage_ent['misclassification_percentage'].round(2)

fig1 = px.bar(group_summary_sorted_percentage_ent.head(num_displayed), x='owner_entity_group', y='misclassification_percentage',
              text='misclassification_percentage',
              labels={'misclassification_percentage': 'Misclassification Percentage (%)', 'owner_entity_group': 'Entity Group'},
              title=f'Top {num_displayed} Entity Groups with higher percentage of misclassification')
fig1.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig1.update_layout(xaxis={'categoryorder': 'total descending'}, yaxis_title="Misclassification Percentage", yaxis_tickformat='.2f')

group_summary_sorted_occurrences_ent = group_summary.sort_values(by='total_ownerships', ascending=False)
group_summary_sorted_occurrences_ent['text'] = group_summary_sorted_occurrences_ent.apply(lambda row: f"{row['misclassified']} / {row['total_ownerships']}", axis=1)

group_summary_sorted_occurrences_ent['display_text'] = group_summary_sorted_occurrences_ent.apply(
    lambda row: f"{row['misclassified']} di {row['total_ownerships']}", axis=1)

fig1 = px.bar(group_summary_sorted_occurrences_ent.head(num_displayed), x='owner_entity_group', y='misclassification_percentage',
              text='display_text',  
              labels={'misclassification_percentage': 'Misclassification Percentage (%)', 'owner_entity_group': 'Entity Group'},
              title=f'Top {num_displayed} Entity Groups with higher percentage of misclassification')
fig1.update_traces(texttemplate='%{text}', textposition='outside')  
fig1.update_layout(xaxis={'categoryorder': 'total descending'}, yaxis_title="Misclassification Percentage", yaxis_tickformat='.2f')


fig2 = px.bar(group_summary_sorted_occurrences_ent.head(num_displayed), x='owner_entity_group', y='misclassification_percentage',
              text='display_text',  
              labels={'misclassification_percentage': 'Misclassification Percentage (%)', 'owner_entity_group': 'Entity Group', 'total_ownerships': 'Total Ownerships'},
              title=f'Percentage of misclassification of top {num_displayed} Entity Groups by number ownerships')
fig2.update_traces(texttemplate='%{text}', textposition='outside')

fig1.update_traces(texttemplate='%{y}', textposition='inside')
fig2.update_traces(texttemplate='%{y}', textposition='inside')
fig1.update_layout(height=650)
fig2.update_layout(height=650)
fig1.show()
fig2.show()


#### Total misclassifications (Entities)

In [146]:
misclassified_df = misclassification_entities_df[misclassification_entities_df['owner_code'].str.startswith('ent')].copy()
misclassified_df['classification_status'] = misclassified_df['misclassified'].map({True: 'Misclassified', False: 'Correct'})

classification_counts = misclassified_df['classification_status'].value_counts()

fig = px.pie(values=classification_counts.values, 
             names=classification_counts.index, 
             title='Entities: Percentage of Misclassifications')

fig.update_traces(textinfo='percent+label')
fig.update_layout(height=400)
fig.show()


## Distribution of Families

#### Load Catastici 1741 expanded version

The expanded version was obtained by creating one row for every owner. If a parcel is owned by $n$ owners, $n$ rows have been created with the same parcel uidx prefix. For example, if parcel 123 is owned by two people, rows with uidx 123_0 and 123_1 are created with the information of the respective owners. Furthermore, a new column `parcel_portion` has been cerated. This attribute is a number from 0 to 1 representing the portion of the parcel owned by this owner. In case of parcel 123 owned by two people this value is 0.5 for each owner.

In [147]:
df_families = pd.read_json('../data_catastici/data_post-processing/pipeline_steps/catastici_1741_STD_expanded.json', dtype={'uidx': str})

In [148]:
min_occurrences = 1
num_displayed = 20

In [149]:
owner_code_mapping = {
    'PPL': 'Person',
    'PPL_m': 'Person (model output)',
    'PPL_VRF': 'Person (to verify)'
}

### Distribution of People Owner Codes

In [150]:
df_families['owner_category'] = df_families['owner_code'].map(lambda x: owner_code_mapping.get(x, 'Person (other)') if 'PPL' in x else 'Non-Person').copy()
df_families = df_families[df_families['owner_code'].str.contains('PPL')]

category_counts = df_families['owner_category'].value_counts().reset_index()
category_counts.columns = ['owner_category', 'count']

pie_fig = px.pie(category_counts, names='owner_category', values='count', 
                 title='Distribution of Owner Codes for People')
pie_fig.update_layout(height=400)
pie_fig.show()


### Distribution of Family ownership

#### Distribution of Family mentions

In [151]:
family_counts_df = df_families.groupby('owner_family_name')['parcel_portion'].sum().reset_index().rename(columns={'parcel_portion': 'weighted_count'})
family_counts_df = family_counts_df.sort_values(by='weighted_count', ascending=False)

filtered_family_counts = family_counts_df[family_counts_df['weighted_count'] >= min_occurrences]
top_families = filtered_family_counts.head(num_displayed)
top_families = top_families[top_families['owner_family_name'] != '']

hist_fig = px.bar(top_families, x='owner_family_name', y='weighted_count',
                  labels={'weighted_count': 'Weighted Number of Occurrences', 'owner_family_name': 'Family Group'}, 
                  title=f'Distribution of Family Ownership - Weighted by Ownership Portion')

hist_fig.update_layout(height=500)
hist_fig.update_traces(texttemplate='%{y}', textposition='inside')
hist_fig.show()

#### Distribution of standardised Family Groups

In [152]:
family_counts_df = df_families.groupby('owner_family_group')['parcel_portion'].sum().reset_index().rename(columns={'parcel_portion': 'weighted_count'})
family_counts_df = family_counts_df.sort_values(by='weighted_count', ascending=False)

filtered_family_counts = family_counts_df[family_counts_df['weighted_count'] >= min_occurrences]
top_families = filtered_family_counts.head(num_displayed)
top_families = top_families[top_families['owner_family_group'] != '']

hist_fig = px.bar(top_families, x='owner_family_group', y='weighted_count',
                  labels={'weighted_count': 'Weighted Number of Occurrences', 'owner_family_group': 'Family Group'}, 
                  title=f'Distribution of Family Ownership - Weighted by Ownership Portion')

hist_fig.update_layout(height=500)
hist_fig.update_traces(texttemplate='%{y}', textposition='inside')
hist_fig.show()


### Misclassifications: Families

#### Number of misclassifications in non-grouped mentions (pre-standardisation)

According to most common mention (not to Family group name).

In [153]:
most_common_names = df_families.groupby('owner_family_group')['owner_family_name'].agg(
    lambda x: x.mode()[0] if not x.mode().empty else pd.NA
).reset_index().rename(columns={'owner_family_name': 'most_common_name'})
df_families = df_families.merge(most_common_names, on='owner_family_group')

df_families['misclassified'] = df_families['owner_family_name'] != df_families['most_common_name']

group_summary_families = df_families.groupby('owner_family_group').agg(
    total_ownerships=('owner_family_group', 'size'),  # total occurrences of each family group
    misclassified=('misclassified', 'sum')  # sum of misclassified instances per group
).reset_index()

group_summary_families['misclassification_percentage'] = (group_summary_families['misclassified'] / group_summary_families['total_ownerships']) * 100
group_summary_families['misclassification_percentage'] = group_summary_families['misclassification_percentage'].round(2)


In [154]:
group_summary_families_misclassified = group_summary_families.sort_values(by='misclassified', ascending=False)

fig1 = px.bar(group_summary_families_misclassified.head(num_displayed), x='owner_family_group', y='misclassified',
              text='misclassified',
              labels={'misclassified': 'Number of Misclassifications', 'owner_family_group': 'Family Group'},
              title=f'Top {num_displayed} Family Groups with higher number of Misclassifications')
fig1.update_traces(texttemplate='%{text}', textposition='outside')
fig1.update_layout(xaxis={'categoryorder': 'total descending'}, yaxis_title="Number of Misclassifications")


group_summary_families_sorted_occurrences = group_summary_families.sort_values(by='total_ownerships', ascending=False)

fig2 = px.bar(group_summary_families_sorted_occurrences.head(num_displayed), x='owner_family_group', y='misclassified',
              text='misclassified',
              labels={'misclassified': 'Number of Misclassifications', 'owner_family_group': 'Family Group', 'total_ownerships': 'Total Ownerships'},
              title=f'Number of Misclassifications of Top {num_displayed} Family Groups by Number of Ownerships')
fig2.update_traces(texttemplate='%{text}', textposition='outside')

fig1.update_layout(height=500)
fig2.update_layout(height=500)
fig1.update_traces(texttemplate='%{y}', textposition='inside')
fig2.update_traces(texttemplate='%{y}', textposition='inside')
fig1.show()
fig2.show()

#### Percentage (%) of misclassifications in non-grouped mentions (pre-standardisation)

In [155]:
group_summary_families_sorted_percentage = group_summary_families.sort_values(by='misclassification_percentage', ascending=False)
group_summary_families_sorted_percentage['misclassification_percentage'] = group_summary_families_sorted_percentage['misclassification_percentage'].round(2)

fig1 = px.bar(group_summary_families_sorted_percentage.head(num_displayed), x='owner_family_group', y='misclassification_percentage',
              text='misclassification_percentage',
              labels={'misclassification_percentage': 'Misclassification Percentage (%)', 'owner_family_group': 'family Group'},
              title=f'Top {num_displayed} Family Groups with higher percentage of misclassification')
fig1.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig1.update_layout(xaxis={'categoryorder': 'total descending'}, yaxis_title="Misclassification Percentage", yaxis_tickformat='.2f')

group_summary_families_sorted_occurrences = group_summary_families.sort_values(by='total_ownerships', ascending=False)
group_summary_families_sorted_occurrences['text'] = group_summary_families_sorted_occurrences.apply(lambda row: f"{row['misclassified']} / {row['total_ownerships']}", axis=1)

group_summary_families_sorted_occurrences['display_text'] = group_summary_families_sorted_occurrences.apply(
    lambda row: f"{row['misclassified']} di {row['total_ownerships']}", axis=1)

fig1 = px.bar(group_summary_families_sorted_occurrences.head(num_displayed), x='owner_family_group', y='misclassification_percentage',
              text='display_text',  
              labels={'misclassification_percentage': 'Misclassification Percentage (%)', 'owner_family_group': 'family Group'},
              title=f'Top {num_displayed} Family Groups with higher percentage of misclassification')
fig1.update_traces(texttemplate='%{text}', textposition='outside')  
fig1.update_layout(xaxis={'categoryorder': 'total descending'}, yaxis_title="Misclassification Percentage", yaxis_tickformat='.2f')


fig2 = px.bar(group_summary_families_sorted_occurrences.head(num_displayed), x='owner_family_group', y='misclassification_percentage',
              text='display_text',  
              labels={'misclassification_percentage': 'Misclassification Percentage (%)', 'owner_family_group': 'family Group', 'total_ownerships': 'Total Ownerships'},
              title=f'Percentage of misclassification of top {num_displayed} Family Groups by number ownerships')
fig2.update_traces(texttemplate='%{text}', textposition='outside')

fig1.update_traces(texttemplate='%{y}', textposition='inside')
fig2.update_traces(texttemplate='%{y}', textposition='inside')
fig1.update_layout(height=500)
fig2.update_layout(height=500)
fig1.show()
fig2.show()


#### Total misclassifications (Families)

In [156]:
df_families['classification_status'] = df_families['misclassified'].map({True: 'Misclassified', False: 'Correct'})
classification_counts = df_families['classification_status'].value_counts()

fig = px.pie(values=classification_counts.values, 
             names=classification_counts.index, 
             title='Families: Percentage of Misclassifications')

fig.update_traces(textinfo='percent+label')
fig.update_layout(height=400)
fig.show()


### Inter-family parcel sharing

#### Couples of family groups sharing most parcels

In [157]:
from itertools import combinations
from collections import Counter
import pandas as pd
import plotly.express as px

df_families['parcel_id'] = df_families['uidx'].apply(lambda x: x.split('_')[0])

shared_parcels = df_families.groupby('parcel_id')['owner_family_group'].apply(list).reset_index()
shared_parcels = shared_parcels[shared_parcels['owner_family_group'].apply(len) > 1]

def create_couples(owner_list):
    return [' - '.join(sorted(couple)) for couple in combinations(owner_list, 2) if couple[0] != couple[1] and couple[0] != '-' and couple[1] != '-']

shared_parcels['couples'] = shared_parcels['owner_family_group'].apply(create_couples)

all_couples = Counter([couple for sublist in shared_parcels['couples'] for couple in sublist])

couples_df = pd.DataFrame(all_couples.items(), columns=['couple', 'shared_parcels'])
sorted_couples_df = couples_df.sort_values(by='shared_parcels', ascending=False)

fig = px.bar(sorted_couples_df.head(num_displayed), x='couple', y='shared_parcels',
             title=f'Top {num_displayed} Family Group Pairs by Shared Parcels',
             labels={'shared_parcels': 'Number of Shared Parcels', 'couple': 'Family Group Pair'})
fig.show()


#### Individual Families with most shared parcels

In [158]:
from collections import Counter

def count_individual_families(shared_parcels):
    individual_families = []
    
    for owner_list in shared_parcels['owner_family_group']:
        for couple in combinations(owner_list, 2):
            if couple[0] != couple[1] and couple[0] != '-' and couple[1] != '-':
                individual_families.extend(couple)
                
    return Counter(individual_families)

family_counts = count_individual_families(shared_parcels)

families_df = pd.DataFrame(family_counts.items(), columns=['Family', 'Count'])
sorted_families_df = families_df.sort_values(by='Count', ascending=False)

fig = px.bar(sorted_families_df.head(num_displayed), x='Family', y='Count',
             title='Top Individual Families with Most Shared Parcels',
             labels={'Count': 'Number of Shared Parcels', 'Family': 'Family'})
fig.show()


### People's Titles and Mestieri

#### Titles

In [159]:
# TODO

#### Mestieri

In [160]:
# TODO